In [1]:
import os

In [2]:
%pwd

'd:\\StyleConsult\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\StyleConsult'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_input_shape: list
    params_classes: int

In [6]:
from gender_classifier.constants import *
from gender_classifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_input_shape=self.params.input_shape,
            params_classes=self.params.classes
        )

        return prepare_base_model_config

In [8]:
import os
from pathlib import Path
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras import regularizers, optimizers

In [9]:
class PrepareBaseModel:
    def __init__(self, config):
        self.config = config

    def get_base_model(self):
        num_classes = self.config.params_classes
        input_shape = self.config.params_input_shape
        
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape,
                                kernel_regularizer=regularizers.l2(0.001), padding="valid"))
        model.add(MaxPooling2D((2, 2)))

        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))

        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))

        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

        self.save_model(path=self.config.base_model_path, model=model)
        model.summary()

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [10]:
try:
    config_manager = ConfigurationManager()
    prepare_base_model_config = config_manager.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

[2024-05-22 01:51:44,758: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-22 01:51:44,761: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-22 01:51:44,763: INFO: common: created directory at: artifacts]


[2024-05-22 01:51:44,764: INFO: common: created directory at: artifacts/prepare_base_model]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 124, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 59, 64)        36928     
                              